In [1]:
import pickle
from sklearn.metrics import confusion_matrix
import requests, bs4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from csv import writer
import requests, bs4
import re
%matplotlib inline
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import os
%load_ext Cython

In [10]:
db = pd.read_csv('2017_2018stats.csv')

In [11]:
db = db[db['Season'] == '2016-2017'].reset_index(drop = True)

In [12]:
db.to_csv('2017stats.csv')

Adding previous seasons to the DF using previously written script:

In [2]:
def get_4fac(team, year):
    
    team_abbrevs = {
    'Cleveland Cavaliers': 'CLE',
    'Boston Celtics':'BOS',
    'Washington Wizards':'WAS',
    'Charlotte Hornets':'CHO',
    'Minnesota Timberwolves':'MIN',
    'Dallas Mavericks':'DAL',
    'Milwaukee Bucks':'MIL',
    'Philadelphia 76ers':'PHI',
    'Phoenix Suns':'PHO',
    'Los Angeles Lakers':'LAL',
    'Utah Jazz':'UTA',
    'Sacramento Kings':'SAC',
    'New York Knicks':'NYK',
    'New Orleans Pelicans':'NOP',
    'Detroit Pistons':'DET',
    'Atlanta Hawks':'ATL',
    'Chicago Bulls':'CHI',
    'Miami Heat':'MIA',
    'Memphis Grizzlies':'MEM',
    'Golden State Warriors':'GSW',
    'Denver Nuggets':'DEN',
    'Brooklyn Nets':'BRK',
    'Los Angeles Clippers':'LAC',
    'Portland Trail Blazers':'POR',
    'Indiana Pacers':'IND',
    'San Antonio Spurs':'SAS',
    'Houston Rockets':'HOU',
    'Oklahoma City Thunder':'OKC',
    'Toronto Raptors':'TOR',
    'Orlando Magic':'ORL'
    }
    
    # Formats webscraped dates to match bball reference URLs for later scraping
    def date_formatter(date):

        if len(str(date.month)) != 2:
            month = '0'+ str(date.month)
        else:
            month = str(date.month)
        
        if len(str(date.day)) != 2:
            day = '0' + str(date.day)
        else:
            day = str(date.day)
        
        newdate = str(date.year) + month + day
        return newdate


    dates = []
    efgs = []
    tovpct = []
    orbpct = []
    ftr = []

        
    # Get team page
    url = f'https://www.basketball-reference.com/teams/{team}/{year}_games.html'
    res = requests.get(url, 'html_parser')
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
        
    # Get just regular season stats
    reg_season = soup.find('table',{'id':'games'})
        
    # Pull regular season game dates, generate list, turn it into datetime objects
    datesoup = reg_season.find_all(attrs = {'data-stat':'date_game'})
    gamedates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date' ]
    game_dates = [datetime.strptime(date, '%a, %b %d, %Y') for date in gamedates]
        
        
    # Find regular season game locations, translate to 1 = home, 0 = away
    souplocs = reg_season.find_all(attrs = {'data-stat':'game_location'})
    locs = [entry.get_text() for entry in souplocs if entry.get_text() == '' or entry.get_text() == '@']
    game_locs = list(map(lambda x: 1 if x =='' else 0, locs))
    
    # Find opponent team names
    oppts = reg_season.find_all(attrs = {'data-stat':'opp_name'})
    opps = [entry.get_text() for entry in oppts if entry.get_text() != "Opponent"]
    opp_abbrev = [team_abbrevs[opp] for opp in opps]
        
    # Get Results
    soupresults = reg_season.find_all(attrs = {'data-stat':'game_result'})
    results = [entry.get_text() for entry in soupresults if entry.get_text() == 'W' or entry.get_text() == 'L']
    rslts = list(map(lambda x: 1 if x =='W' else 0, results))

    # Get game stats
    for i, gamedate in enumerate(game_dates):
            
        date = date_formatter(gamedate)
        dates.append(date)
            
        if game_locs[i] == 1:
            url = f'https://www.basketball-reference.com/boxscores/{date}0{team}.html'
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
            fourfac = soup.find('div',{'id':'all_four_factors'})
            efgs.append(fourfac.find_all('td',{'data-stat':'efg_pct'})[1].get_text())
            tovpct.append(fourfac.find_all('td',{'data-stat':'tov_pct'})[1].get_text())
            orbpct.append(fourfac.find_all('td',{'data-stat':'orb_pct'})[1].get_text())
            ftr.append(fourfac.find_all('td',{'data-stat':'ft_rate'})[1].get_text())
        else:
            url = f'https://www.basketball-reference.com/boxscores/{date}0{team_abbrevs[opps[i]]}.html'
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
            fourfac = soup.find('div',{'id':'all_four_factors'})
            efgs.append(fourfac.find_all('td',{'data-stat':'efg_pct'})[0].get_text())
            tovpct.append(fourfac.find_all('td',{'data-stat':'tov_pct'})[0].get_text())
            orbpct.append(fourfac.find_all('td',{'data-stat':'orb_pct'})[0].get_text())
            ftr.append(fourfac.find_all('td',{'data-stat':'ft_rate'})[0].get_text())

        print("Team: " + team + " Date: " + date + " Opponent: " + team_abbrevs[opps[i]] + " Result: " + str(rslts[i]) + " EFG%: " + efgs[i] + " TOV%: " + tovpct[i] + " ORB%: " + orbpct[i] + " FTR: " + ftr[i])
    
    teamlist= [team] * 82
    data = {'Team': teamlist, "Game Number":list(range(1,83)), "Result":rslts, "Date": dates, "Opponent": opp_abbrev, "EFG%": efgs, "TOV%" : tovpct, "ORB%":orbpct, "FTR":ftr}
    df = pd.DataFrame.from_dict(data)
    df['EFG%'] = df['EFG%'].apply(lambda x: float(x))
    df['TOV%'] = df['TOV%'].apply(lambda x: float(x))
    df['ORB%'] = df['ORB%'].apply(lambda x: float(x))
    df['FTR'] = df['FTR'].apply(lambda x: float(x))
    df['TOV%'] = df['TOV%']/100.0
    df['ORB%'] = df['ORB%']/100.0

    
    return df

In [3]:
team_abbrevs = {
    'Cleveland Cavaliers': 'CLE',
    'Boston Celtics':'BOS',
    'Washington Wizards':'WAS',
    'Charlotte Hornets':'CHO',
    'Minnesota Timberwolves':'MIN',
    'Dallas Mavericks':'DAL',
    'Milwaukee Bucks':'MIL',
    'Philadelphia 76ers':'PHI',
    'Phoenix Suns':'PHO',
    'Los Angeles Lakers':'LAL',
    'Utah Jazz':'UTA',
    'Sacramento Kings':'SAC',
    'New York Knicks':'NYK',
    'New Orleans Pelicans':'NOP',
    'Detroit Pistons':'DET',
    'Atlanta Hawks':'ATL',
    'Chicago Bulls':'CHI',
    'Miami Heat':'MIA',
    'Memphis Grizzlies':'MEM',
    'Golden State Warriors':'GSW',
    'Denver Nuggets':'DEN',
    'Brooklyn Nets':'BRK',
    'Los Angeles Clippers':'LAC',
    'Portland Trail Blazers':'POR',
    'Indiana Pacers':'IND',
    'San Antonio Spurs':'SAS',
    'Houston Rockets':'HOU',
    'Oklahoma City Thunder':'OKC',
    'Toronto Raptors':'TOR',
    'Orlando Magic':'ORL'
}
    
df_2017 = pd.DataFrame()

for team in list(team_abbrevs.values()):
    team_db = get_4fac(team,'2017')
    df_2017 = pd.concat([df_2017, team_db])

Team: CLE Date: 20161025 Opponent: NYK Result: 1 EFG%: .548 TOV%: 12.0 ORB%: 27.5 FTR: .149
Team: CLE Date: 20161028 Opponent: TOR Result: 1 EFG%: .494 TOV%: 16.1 ORB%: 21.4 FTR: .203
Team: CLE Date: 20161029 Opponent: ORL Result: 1 EFG%: .472 TOV%: 8.2 ORB%: 26.0 FTR: .250
Team: CLE Date: 20161101 Opponent: HOU Result: 1 EFG%: .614 TOV%: 10.2 ORB%: 36.4 FTR: .313
Team: CLE Date: 20161103 Opponent: BOS Result: 1 EFG%: .571 TOV%: 7.4 ORB%: 19.4 FTR: .381
Team: CLE Date: 20161105 Opponent: PHI Result: 1 EFG%: .484 TOV%: 13.9 ORB%: 21.6 FTR: .154
Team: CLE Date: 20161108 Opponent: ATL Result: 0 EFG%: .429 TOV%: 5.2 ORB%: 25.0 FTR: .212
Team: CLE Date: 20161111 Opponent: WAS Result: 1 EFG%: .500 TOV%: 13.3 ORB%: 26.5 FTR: .221
Team: CLE Date: 20161113 Opponent: CHO Result: 1 EFG%: .500 TOV%: 8.4 ORB%: 25.5 FTR: .087
Team: CLE Date: 20161115 Opponent: TOR Result: 1 EFG%: .608 TOV%: 15.1 ORB%: 26.3 FTR: .241
Team: CLE Date: 20161116 Opponent: IND Result: 0 EFG%: .471 TOV%: 16.0 ORB%: 23.9 FT

In [4]:
df_2017.to_csv('2017stats.csv')

In [5]:
db= df_2017

In [6]:
db.reset_index(inplace = True, drop = True)

In [7]:
db['Game Number'] = db['Game Number'].apply(lambda x: int(x))

In [8]:
db['Season'] = '2016-2017'
db['teamCumEFG%'] = np.zeros(len(db['Team']))
db['teamCumTOV%'] = np.zeros(len(db['Team']))
db['teamCumORB%'] = np.zeros(len(db['Team']))
db['teamCumFTR'] = np.zeros(len(db['Team']))

for i in range(0,len(db['Team'])):
    if db['Game Number'][i] == 1:
        db['teamCumEFG%'][i] = db['EFG%'][i]
    else:
        db['teamCumEFG%'][i] = (db[db['Team'] == db['Team'][i]][:db['Game Number'][i]]['EFG%'].sum()/db['Game Number'][i])
    print("Current team: " + db['Team'][i] + " Game Number: " + str(db['Game Number'][i]) + " Running EFG: " + str(db['teamCumEFG%'][i]))

for i in range(0,len(db['Team'])):
    if db['Game Number'][i] == 1:
        db['teamCumTOV%'][i] = db['TOV%'][i]
    else:
        db['teamCumTOV%'][i] = (db[db['Team'] == db['Team'][i]][:db['Game Number'][i]]['TOV%'].sum()/db['Game Number'][i])
    print("Current team: " + db['Team'][i] + " Game Number: " + str(db['Game Number'][i]) + " Running TOV: " + str(db['teamCumTOV%'][i]))
    
for i in range(0,len(db['Team'])):
    if db['Game Number'][i] == 1:
        db['teamCumORB%'][i] = db['ORB%'][i]
    else:
        db['teamCumORB%'][i] = (db[db['Team'] == db['Team'][i]][:db['Game Number'][i]]['ORB%'].sum()/db['Game Number'][i])
    print("Current team: " + db['Team'][i] + " Game Number: " + str(db['Game Number'][i]) + " Running ORB: " + str(db['teamCumORB%'][i]))

for i in range(0,len(db['Team'])):
    if db['Game Number'][i] == 1:
        db['teamCumFTR'][i] = db['FTR'][i]
    else:
        db['teamCumFTR'][i] = (db[db['Team'] == db['Team'][i]][:db['Game Number'][i]]['FTR'].sum()/db['Game Number'][i])
    print("Current team: " + db['Team'][i] + " Game Number: " + str(db['Game Number'][i]) + " Running FTR: " + str(db['teamCumFTR'][i]))
    
db['Opp EFG%'] = np.zeros(len(db['Team']))
db['Opp TOV%'] = np.zeros(len(db['Team']))
db['Opp ORB%'] = np.zeros(len(db['Team']))
db['Opp FTR'] = np.zeros(len(db['Team']))

for i, team in enumerate(db['Team']):
    db['Opp EFG%'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opponent'] == team)]['EFG%']
    db['Opp TOV%'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opponent'] == team)]['TOV%']
    db['Opp ORB%'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opponent'] == team)]['ORB%']
    db['Opp FTR'][i] = db[(db['Date'] == db['Date'][i]) & (db['Opponent'] == team)]['FTR']
    
db['Opp CumEFG%'] = np.zeros(len(db['Team']))
db['Opp CumTOV%'] = np.zeros(len(db['Team']))
db['Opp CumORB%'] = np.zeros(len(db['Team']))
db['Opp CumFTR'] = np.zeros(len(db['Team']))

for i, opp in enumerate(db['Opponent']):
    db['Opp CumTOV%'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['teamCumTOV%']
    db['Opp CumEFG%'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['teamCumEFG%']
    db['Opp CumORB%'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['teamCumORB%']
    db['Opp CumFTR'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['teamCumFTR']

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Current team: CLE Game Number: 1 Running EFG: 0.548
Current team: CLE Game Number: 2 Running EFG: 0.521
Current team: CLE Game Number: 3 Running EFG: 0.5046666666666667
Current team: CLE Game Number: 4 Running EFG: 0.532
Current team: CLE Game Number: 5 Running EFG: 0.5398
Current team: CLE Game Number: 6 Running EFG: 0.5305
Current team: CLE Game Number: 7 Running EFG: 0.5159999999999999
Current team: CLE Game Number: 8 Running EFG: 0.514
Current team: CLE Game Number: 9 Running EFG: 0.5124444444444445
Current team: CLE Game Number: 10 Running EFG: 0.522
Current team: CLE Game Number: 11 Running EFG: 0.5173636363636364
Current team: CLE Game Number: 12 Running EFG: 0.5199166666666667
Current team: CLE Game Number: 13 Running EFG: 0.5299230769230769
Current team: CLE Game Number: 14 Running EFG: 0.5366428571428571
Current team: CLE Game Number: 15 Running EFG: 0.5327333333333333
Current team: CLE Game Number: 16 Running EFG: 0.5319375
Current team: CLE Game Number: 17 Running EFG: 0.52

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Current team: CLE Game Number: 7 Running TOV: 0.10428571428571429
Current team: CLE Game Number: 8 Running TOV: 0.107875
Current team: CLE Game Number: 9 Running TOV: 0.10522222222222222
Current team: CLE Game Number: 10 Running TOV: 0.10979999999999998
Current team: CLE Game Number: 11 Running TOV: 0.11436363636363635
Current team: CLE Game Number: 12 Running TOV: 0.11541666666666665
Current team: CLE Game Number: 13 Running TOV: 0.11338461538461536
Current team: CLE Game Number: 14 Running TOV: 0.11442857142857142
Current team: CLE Game Number: 15 Running TOV: 0.11206666666666666
Current team: CLE Game Number: 16 Running TOV: 0.116125
Current team: CLE Game Number: 17 Running TOV: 0.11947058823529412
Current team: CLE Game Number: 18 Running TOV: 0.12233333333333334
Current team: CLE Game Number: 19 Running TOV: 0.12257894736842104
Current team: CLE Game Number: 20 Running TOV: 0.12214999999999998
Current team: CLE Game Number: 21 Running TOV: 0.12142857142857141
Current team: CLE Ga

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Current team: CLE Game Number: 2 Running ORB: 0.2445
Current team: CLE Game Number: 3 Running ORB: 0.24966666666666668
Current team: CLE Game Number: 4 Running ORB: 0.27825
Current team: CLE Game Number: 5 Running ORB: 0.26139999999999997
Current team: CLE Game Number: 6 Running ORB: 0.2538333333333333
Current team: CLE Game Number: 7 Running ORB: 0.2532857142857143
Current team: CLE Game Number: 8 Running ORB: 0.25475000000000003
Current team: CLE Game Number: 9 Running ORB: 0.25477777777777777
Current team: CLE Game Number: 10 Running ORB: 0.2556
Current team: CLE Game Number: 11 Running ORB: 0.2540909090909091
Current team: CLE Game Number: 12 Running ORB: 0.24391666666666667
Current team: CLE Game Number: 13 Running ORB: 0.2433846153846154
Current team: CLE Game Number: 14 Running ORB: 0.24335714285714286
Current team: CLE Game Number: 15 Running ORB: 0.24546666666666667
Current team: CLE Game Number: 16 Running ORB: 0.2410625
Current team: CLE Game Number: 17 Running ORB: 0.235941

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Current team: CLE Game Number: 1 Running FTR: 0.149
Current team: CLE Game Number: 2 Running FTR: 0.176
Current team: CLE Game Number: 3 Running FTR: 0.20066666666666666
Current team: CLE Game Number: 4 Running FTR: 0.22875
Current team: CLE Game Number: 5 Running FTR: 0.2592
Current team: CLE Game Number: 6 Running FTR: 0.24166666666666667
Current team: CLE Game Number: 7 Running FTR: 0.2374285714285714
Current team: CLE Game Number: 8 Running FTR: 0.235375
Current team: CLE Game Number: 9 Running FTR: 0.21888888888888888
Current team: CLE Game Number: 10 Running FTR: 0.2211
Current team: CLE Game Number: 11 Running FTR: 0.21372727272727274
Current team: CLE Game Number: 12 Running FTR: 0.209
Current team: CLE Game Number: 13 Running FTR: 0.21
Current team: CLE Game Number: 14 Running FTR: 0.2135
Current team: CLE Game Number: 15 Running FTR: 0.21666666666666667
Current team: CLE Game Number: 16 Running FTR: 0.2223125
Current team: CLE Game Number: 17 Running FTR: 0.23394117647058824


C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\

In [9]:
db.to_csv('2017stats.csv', index = False)

In [14]:
def get_locs():
    team_abbrevs = {
    'Cleveland Cavaliers': 'CLE',
    'Boston Celtics':'BOS',
    'Washington Wizards':'WAS',
    'Charlotte Hornets':'CHO',
    'Minnesota Timberwolves':'MIN',
    'Dallas Mavericks':'DAL',
    'Milwaukee Bucks':'MIL',
    'Philadelphia 76ers':'PHI',
    'Phoenix Suns':'PHO',
    'Los Angeles Lakers':'LAL',
    'Utah Jazz':'UTA',
    'Sacramento Kings':'SAC',
    'New York Knicks':'NYK',
    'New Orleans Pelicans':'NOP',
    'Detroit Pistons':'DET',
    'Atlanta Hawks':'ATL',
    'Chicago Bulls':'CHI',
    'Miami Heat':'MIA',
    'Memphis Grizzlies':'MEM',
    'Golden State Warriors':'GSW',
    'Denver Nuggets':'DEN',
    'Brooklyn Nets':'BRK',
    'Los Angeles Clippers':'LAC',
    'Portland Trail Blazers':'POR',
    'Indiana Pacers':'IND',
    'San Antonio Spurs':'SAS',
    'Houston Rockets':'HOU',
    'Oklahoma City Thunder':'OKC',
    'Toronto Raptors':'TOR',
    'Orlando Magic':'ORL'
    }
    
    all_locs = []
    for team in list(team_abbrevs.values()):
        print("Current team: " + team)
        url = f'https://www.basketball-reference.com/teams/{team}/2017_games.html'
        res = requests.get(url, 'html_parser')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'html.parser')
        
        # Get just regular season 
        reg_season = soup.find('table',{'id':'games'})
        # Find regular season game locations, translate to 1 = home, 0 = away
        souplocs = reg_season.find_all(attrs = {'data-stat':'game_location'})
        locs = [entry.get_text() for entry in souplocs if entry.get_text() == '' or entry.get_text() == '@']
        game_locs = list(map(lambda x: 1 if x =='' else 0, locs))
        all_locs.append(game_locs)
        
    return all_locs
gamelocs = get_locs()
flat_locs = [item for sublist in gamelocs for item in sublist]
locs = {"Location":flat_locs}
locdf = pd.DataFrame.from_dict(locs)
updatedb = pd.concat([db, locdf], axis = 1)
updatedb.to_csv('2017stats.csv', index = False)

Current team: CLE
Current team: BOS
Current team: WAS
Current team: CHO
Current team: MIN
Current team: DAL
Current team: MIL
Current team: PHI
Current team: PHO
Current team: LAL
Current team: UTA
Current team: SAC
Current team: NYK
Current team: NOP
Current team: DET
Current team: ATL
Current team: CHI
Current team: MIA
Current team: MEM
Current team: GSW
Current team: DEN
Current team: BRK
Current team: LAC
Current team: POR
Current team: IND
Current team: SAS
Current team: HOU
Current team: OKC
Current team: TOR
Current team: ORL


In [15]:
db = pd.read_csv('2017stats.csv')

In [18]:
db.drop(['Location.1', 'Unnamed: 0'], axis = 1, inplace = True)

## See adjusted stats below these

In [19]:
## IRRELEVANT

rollingEFG = []
for team in db['Team'].unique():
    teamRollingEFG = db[db['Team'] == team]['EFG%'].rolling(9).mean()
    rollingEFG.append(teamRollingEFG)    
flat_EFG = [item for sublist in rollingEFG for item in sublist]
EFG = {"Rolling9EFG":flat_EFG}
EFG9df = pd.DataFrame.from_dict(EFG)
updatedb = pd.concat([db, EFG9df], axis = 1)

rollingTOV = []
for team in updatedb['Team'].unique():
    teamRollingTOV = updatedb[updatedb['Team'] == team]['TOV%'].rolling(9).mean()
    rollingTOV.append(teamRollingTOV)
    
flat_TOV = [item for sublist in rollingTOV for item in sublist]
TOV = {"Rolling9TOV":flat_TOV}
TOV9df = pd.DataFrame.from_dict(TOV)
db = pd.concat([updatedb, TOV9df], axis = 1)

rollingORB = []
for team in db['Team'].unique():
    teamRollingORB = db[db['Team'] == team]['ORB%'].rolling(9).mean()
    rollingORB.append(teamRollingORB)
    
flat_ORB = [item for sublist in rollingORB for item in sublist]
ORB = {"Rolling9ORB":flat_ORB}
ORB9df = pd.DataFrame.from_dict(ORB)
db = pd.concat([db, ORB9df], axis = 1)

rollingFTR = []
for team in db['Team'].unique():
    teamRollingFTR = db[db['Team'] == team]['FTR'].rolling(9).mean()
    rollingFTR.append(teamRollingFTR)
    
flat_FTR = [item for sublist in rollingFTR for item in sublist]
FTR = {"Rolling9FTR":flat_FTR}
FTR9df = pd.DataFrame.from_dict(FTR)
db = pd.concat([db, FTR9df], axis = 1)

In [20]:
## IRRELEVANT

db['Opp R9EFG'] = np.zeros(len(db['Team']))
db['Opp R9TOV'] = np.zeros(len(db['Team']))
db['Opp R9ORB'] = np.zeros(len(db['Team']))
db['Opp R9FTR'] = np.zeros(len(db['Team']))

for i, opp in enumerate(db['Opponent']):
    db['Opp R9EFG'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9EFG']
    db['Opp R9TOV'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9TOV']
    db['Opp R9ORB'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9ORB']
    db['Opp R9FTR'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9FTR']

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\helgeson\AppData\Lo

In [21]:
db.to_csv('2017stats.csv', index = False)

In [22]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Data columns (total 31 columns):
Team           2460 non-null object
Game Number    2460 non-null int64
Result         2460 non-null int64
Date           2460 non-null int64
Opponent       2460 non-null object
EFG%           2460 non-null float64
TOV%           2460 non-null float64
ORB%           2460 non-null float64
FTR            2460 non-null float64
Season         2460 non-null object
teamCumEFG%    2460 non-null float64
teamCumTOV%    2460 non-null float64
teamCumORB%    2460 non-null float64
teamCumFTR     2460 non-null float64
Opp EFG%       2460 non-null float64
Opp TOV%       2460 non-null float64
Opp ORB%       2460 non-null float64
Opp FTR        2460 non-null float64
Opp CumEFG%    2460 non-null float64
Opp CumTOV%    2460 non-null float64
Opp CumORB%    2460 non-null float64
Opp CumFTR     2460 non-null float64
Location       2460 non-null int64
Rolling9EFG    2220 non-null float64
Rolling9TOV    2

In [27]:
db2019 = pd.read_csv('2019stats.csv')
db2018 = pd.read_csv('2018stats.csv')
db2017 = pd.read_csv('2017stats.csv')

In [28]:
db = pd.concat([db2019, db2018, db2017])

In [29]:
db.to_csv("2017")

,Team,Game Number,Location,Result,Date,Season,Opponent,EFG%,TOV%,ORB%,...,Opp CumORB%,Opp CumFTR,Rolling9EFG,Rolling9TOV,Rolling9ORB,Rolling9FTR,Opp R9EFG,Opp R9TOV,Opp R9ORB,Opp R9FTR
0,CLE,1,0,0,20181017,2018-2019,TOR,0.441,0.135,0.286,...,0.186000,0.1300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CLE,2,0,0,20181019,2018-2019,MIN,0.527,0.117,0.295,...,0.264500,0.3215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CLE,3,1,0,20181021,2018-2019,ATL,0.515,0.146,0.300,...,0.205333,0.1910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CLE,4,1,0,20181024,2018-2019,BRK,0.399,0.135,0.308,...,0.219250,0.1910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CLE,5,0,0,20181025,2018-2019,DET,0.506,0.086,0.167,...,0.266400,0.1918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def get_pace(teams,opps, home, dates):

    all_pace = []
    for i, date in enumerate(dates):
        team = teams[i]
        opp = opps[i]
        if home[i] == 1:
        
            url = f'https://www.basketball-reference.com/boxscores/{date}0{team}.html'
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
            pace = soup.find('div',{'id':'all_four_factors'})
            all_pace.append(pace.find_all('td',{'data-stat':'pace'})[0].get_text())
            print("Current team: ", team, " Current Opponent: ", opp, " Pace: ", pace.find_all('td',{'data-stat':'pace'})[0].get_text())
        else:

            url = f'https://www.basketball-reference.com/boxscores/{date}0{opp}.html'
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
            pace = soup.find('div',{'id':'all_four_factors'})
            all_pace.append(pace.find_all('td',{'data-stat':'pace'})[0].get_text())
            print("Current team: ", team, " Current Opponent: ", opp, " Pace: ", pace.find_all('td',{'data-stat':'pace'})[0].get_text())
    return all_pace

gamepace = get_pace(db['Team'],db['Opponent'],db['Location'],db['Date'])
pace = {"Pace":gamepace}
pacedb = pd.DataFrame.from_dict(pace)


Current team:  CLE  Current Opponent:  NYK  Pace:  99.9
Current team:  CLE  Current Opponent:  TOR  Pace:  93.9
Current team:  CLE  Current Opponent:  ORL  Pace:  96.5
Current team:  CLE  Current Opponent:  HOU  Pace:  92.8
Current team:  CLE  Current Opponent:  BOS  Pace:  99.4
Current team:  CLE  Current Opponent:  PHI  Pace:  102.3
Current team:  CLE  Current Opponent:  ATL  Pace:  97.6
Current team:  CLE  Current Opponent:  WAS  Pace:  97.4
Current team:  CLE  Current Opponent:  CHO  Pace:  92.5
Current team:  CLE  Current Opponent:  TOR  Pace:  99.0
Current team:  CLE  Current Opponent:  IND  Pace:  97.1
Current team:  CLE  Current Opponent:  DET  Pace:  95.3
Current team:  CLE  Current Opponent:  POR  Pace:  98.3
Current team:  CLE  Current Opponent:  DAL  Pace:  96.7
Current team:  CLE  Current Opponent:  PHI  Pace:  96.6
Current team:  CLE  Current Opponent:  MIL  Pace:  99.6
Current team:  CLE  Current Opponent:  LAC  Pace:  94.7
Current team:  CLE  Current Opponent:  CHI  Pac

In [6]:
db = pd.concat([db, pacedb], axis = 1)

In [7]:
## IRRELEVANT

## Add 3 game rolling Four Factors

############TOV ###############
rollingTOV = []
for team in db['Team'].unique():
    teamRollingTOV = db[db['Team'] == team]['TOV%'].rolling(3).mean()
    rollingTOV.append(teamRollingTOV)
    
flat_TOV = [item for sublist in rollingTOV for item in sublist]
TOV = {"rolling3TOV":flat_TOV}
TOV3df = pd.DataFrame.from_dict(TOV)
db = pd.concat([db, TOV3df], axis = 1)

############### ORB ###############
rollingORB = []
for team in db['Team'].unique():
    teamRollingORB = db[db['Team'] == team]['ORB%'].rolling(3).mean()
    rollingORB.append(teamRollingORB)
    
flat_ORB = [item for sublist in rollingORB for item in sublist]
ORB = {"rolling3ORB":flat_ORB}
ORB3df = pd.DataFrame.from_dict(ORB)
db = pd.concat([db, ORB3df], axis = 1)

############ FTR ###############
rollingFTR = []
for team in db['Team'].unique():
    teamRollingFTR = db[db['Team'] == team]['FTR'].rolling(3).mean()
    rollingFTR.append(teamRollingFTR)
    
flat_FTR = [item for sublist in rollingFTR for item in sublist]
FTR = {"rolling3FTR":flat_FTR}
FTR3df = pd.DataFrame.from_dict(FTR)
db = pd.concat([db, FTR3df], axis = 1)

############## EFG #############
rollingEFG = []
for team in db['Team'].unique():
    teamRollingEFG = db[db['Team'] == team]['EFG%'].rolling(3).mean()
    rollingEFG.append(teamRollingEFG)
    
flat_EFG = [item for sublist in rollingEFG for item in sublist]
EFG = {"rolling3EFG":flat_EFG}
EFG3df = pd.DataFrame.from_dict(EFG)
db = pd.concat([db, EFG3df], axis = 1)

###################################
# 9 and 3 game rolling pace
##############################

rollingPace = []
for team in db['Team'].unique():
    teamRollingPace = db[db['Team'] == team]['Pace'].rolling(9).mean()
    rollingPace.append(teamRollingPace)
    
flat_pace = [item for sublist in rollingPace for item in sublist]
pace9 = {"rolling9pace":flat_pace}
Pace9df = pd.DataFrame.from_dict(pace9)
db = pd.concat([db, Pace9df], axis = 1)

rollingPace = []
for team in db['Team'].unique():
    teamRollingPace = db[db['Team'] == team]['Pace'].rolling(3).mean()
    rollingPace.append(teamRollingPace)
    
flat_pace = [item for sublist in rollingPace for item in sublist]
pace3 = {"rolling3pace":flat_pace}
Pace3df = pd.DataFrame.from_dict(pace3)
db = pd.concat([db, Pace3df], axis = 1)

In [ ]:
## IRRELEVANT

db['Opp R3EFG'] = np.zeros(len(db['Team']))
db['Opp R3TOV'] = np.zeros(len(db['Team']))
db['Opp R3ORB'] = np.zeros(len(db['Team']))
db['Opp R3FTR'] = np.zeros(len(db['Team']))
db['Opp R3Pace'] = np.zeros(len(db['Team']))
db['Opp R9Pace'] = np.zeros(len(db['Team']))

for i, opp in enumerate(db['Opponent']):
    db['Opp R3EFG'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['rolling3EFG']
    db['Opp R3TOV'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['rolling3TOV']
    db['Opp R3ORB'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['rolling3ORB']
    db['Opp R3FTR'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['rolling3FTR']
    db['Opp R3Pace'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['rolling3pace']
    db['Opp R9Pace'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['rolling9pace']

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-

In [ ]:
db.to_csv('2017stats.csv', index = False)

## ADJUSTING STATS 

In [3]:
### Season cumulative stats

for i in range(0,len(db['Team'])):
    if db['Game Number'][i] == 1:
        db['teamCumEFG%'][i] = float('nan')
    else:
        db['teamCumEFG%'][i] = (db[db['Team'] == db['Team'][i]][:db['Game Number'][i-1]]['EFG%'].sum()/db['Game Number'][i-1])
    print("Current team: " + db['Team'][i] + " Game Number: " + str(db['Game Number'][i]) + " Running EFG: " + str(db['teamCumEFG%'][i]))
    
for i in range(0,len(db['Team'])):
    if db['Game Number'][i] == 1:
        db['teamCumTOV%'][i] = float('nan')
    else:
        db['teamCumTOV%'][i] = (db[db['Team'] == db['Team'][i]][:db['Game Number'][i-1]]['TOV%'].sum()/db['Game Number'][i-1])
    print("Current team: " + db['Team'][i] + " Game Number: " + str(db['Game Number'][i]) + " Running TOV: " + str(db['teamCumTOV%'][i]))
    
for i in range(0,len(db['Team'])):
    if db['Game Number'][i] == 1:
        db['teamCumORB%'][i] = float('nan')
    else:
        db['teamCumORB%'][i] = (db[db['Team'] == db['Team'][i]][:db['Game Number'][i-1]]['ORB%'].sum()/db['Game Number'][i-1])
    print("Current team: " + db['Team'][i] + " Game Number: " + str(db['Game Number'][i]) + " Running ORB: " + str(db['teamCumORB%'][i]))
    

for i in range(0,len(db['Team'])):
    if db['Game Number'][i] == 1:
        db['teamCumFTR'][i] = float('nan')
    else:
        db['teamCumFTR'][i] = (db[db['Team'] == db['Team'][i]][:db['Game Number'][i-1]]['FTR'].sum()/db['Game Number'][i-1])
    print("Current team: " + db['Team'][i] + " Game Number: " + str(db['Game Number'][i]) + " Running FTR: " + str(db['teamCumFTR'][i]))

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Current team: CLE Game Number: 1 Running EFG: nan
Current team: CLE Game Number: 2 Running EFG: 0.5479999999999999
Current team: CLE Game Number: 3 Running EFG: 0.5209999999999999
Current team: CLE Game Number: 4 Running EFG: 0.5046666666666666
Current team: CLE Game Number: 5 Running EFG: 0.5319999999999999
Current team: CLE Game Number: 6 Running EFG: 0.5398
Current team: CLE Game Number: 7 Running EFG: 0.5305
Current team: CLE Game Number: 8 Running EFG: 0.5159999999999999
Current team: CLE Game Number: 9 Running EFG: 0.514
Current team: CLE Game Number: 10 Running EFG: 0.5124444444444445
Current team: CLE Game Number: 11 Running EFG: 0.522
Current team: CLE Game Number: 12 Running EFG: 0.5173636363636364
Current team: CLE Game Number: 13 Running EFG: 0.5199166666666667
Current team: CLE Game Number: 14 Running EFG: 0.5299230769230769
Current team: CLE Game Number: 15 Running EFG: 0.5366428571428571
Current team: CLE Game Number: 16 Running EFG: 0.5327333333333333
Current team: CLE 

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Current team: CLE Game Number: 10 Running TOV: 0.10522222222222222
Current team: CLE Game Number: 11 Running TOV: 0.10979999999999998
Current team: CLE Game Number: 12 Running TOV: 0.11436363636363635
Current team: CLE Game Number: 13 Running TOV: 0.11541666666666665
Current team: CLE Game Number: 14 Running TOV: 0.11338461538461536
Current team: CLE Game Number: 15 Running TOV: 0.11442857142857142
Current team: CLE Game Number: 16 Running TOV: 0.11206666666666666
Current team: CLE Game Number: 17 Running TOV: 0.116125
Current team: CLE Game Number: 18 Running TOV: 0.11947058823529412
Current team: CLE Game Number: 19 Running TOV: 0.12233333333333334
Current team: CLE Game Number: 20 Running TOV: 0.12257894736842104
Current team: CLE Game Number: 21 Running TOV: 0.12214999999999998
Current team: CLE Game Number: 22 Running TOV: 0.12142857142857141
Current team: CLE Game Number: 23 Running TOV: 0.12345454545454544
Current team: CLE Game Number: 24 Running TOV: 0.12491304347826086
Curren

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Current team: CLE Game Number: 3 Running ORB: 0.2445
Current team: CLE Game Number: 4 Running ORB: 0.24966666666666668
Current team: CLE Game Number: 5 Running ORB: 0.27825
Current team: CLE Game Number: 6 Running ORB: 0.26139999999999997
Current team: CLE Game Number: 7 Running ORB: 0.2538333333333333
Current team: CLE Game Number: 8 Running ORB: 0.2532857142857143
Current team: CLE Game Number: 9 Running ORB: 0.25475000000000003
Current team: CLE Game Number: 10 Running ORB: 0.25477777777777777
Current team: CLE Game Number: 11 Running ORB: 0.2556
Current team: CLE Game Number: 12 Running ORB: 0.2540909090909091
Current team: CLE Game Number: 13 Running ORB: 0.24391666666666667
Current team: CLE Game Number: 14 Running ORB: 0.2433846153846154
Current team: CLE Game Number: 15 Running ORB: 0.24335714285714286
Current team: CLE Game Number: 16 Running ORB: 0.24546666666666667
Current team: CLE Game Number: 17 Running ORB: 0.2410625
Current team: CLE Game Number: 18 Running ORB: 0.23594

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Current team: CLE Game Number: 5 Running FTR: 0.22875
Current team: CLE Game Number: 6 Running FTR: 0.2592
Current team: CLE Game Number: 7 Running FTR: 0.24166666666666667
Current team: CLE Game Number: 8 Running FTR: 0.2374285714285714
Current team: CLE Game Number: 9 Running FTR: 0.235375
Current team: CLE Game Number: 10 Running FTR: 0.21888888888888888
Current team: CLE Game Number: 11 Running FTR: 0.2211
Current team: CLE Game Number: 12 Running FTR: 0.21372727272727274
Current team: CLE Game Number: 13 Running FTR: 0.209
Current team: CLE Game Number: 14 Running FTR: 0.21
Current team: CLE Game Number: 15 Running FTR: 0.2135
Current team: CLE Game Number: 16 Running FTR: 0.21666666666666667
Current team: CLE Game Number: 17 Running FTR: 0.2223125
Current team: CLE Game Number: 18 Running FTR: 0.23394117647058824
Current team: CLE Game Number: 19 Running FTR: 0.22427777777777777
Current team: CLE Game Number: 20 Running FTR: 0.2262631578947368
Current team: CLE Game Number: 21 Ru

In [4]:
### Opponent cumulative stats

for i, opp in enumerate(db['Opponent']):
    print("Teams left:", str(len(db['Opponent'])-i))
    db['Opp CumTOV%'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['teamCumTOV%']
    db['Opp CumEFG%'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['teamCumEFG%']
    db['Opp CumORB%'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['teamCumORB%']
    db['Opp CumFTR'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['teamCumFTR']

Teams left: 2460
Teams left: 2459


C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\helgeson\AppData\Local\Continuum\anaconda3

Teams left: 2458
Teams left: 2457
Teams left: 2456
Teams left: 2455
Teams left: 2454
Teams left: 2453
Teams left: 2452
Teams left: 2451
Teams left: 2450
Teams left: 2449
Teams left: 2448
Teams left: 2447
Teams left: 2446
Teams left: 2445
Teams left: 2444
Teams left: 2443
Teams left: 2442
Teams left: 2441
Teams left: 2440
Teams left: 2439
Teams left: 2438
Teams left: 2437
Teams left: 2436
Teams left: 2435
Teams left: 2434
Teams left: 2433
Teams left: 2432
Teams left: 2431
Teams left: 2430
Teams left: 2429
Teams left: 2428
Teams left: 2427
Teams left: 2426
Teams left: 2425
Teams left: 2424
Teams left: 2423
Teams left: 2422
Teams left: 2421
Teams left: 2420
Teams left: 2419
Teams left: 2418
Teams left: 2417
Teams left: 2416
Teams left: 2415
Teams left: 2414
Teams left: 2413
Teams left: 2412
Teams left: 2411
Teams left: 2410
Teams left: 2409
Teams left: 2408
Teams left: 2407
Teams left: 2406
Teams left: 2405
Teams left: 2404
Teams left: 2403
Teams left: 2402
Teams left: 2401
Teams left: 24

In [5]:
### 9 game rolling averages

EFG9DB = pd.DataFrame()
rollingEFG = []
for team in db['Team'].unique():
    teamRollingEFG = db[db['Team'] == team]['EFG%'].rolling(9).mean()
    rollingEFG.append(teamRollingEFG)
    EFG = {"Rolling9EFG":teamRollingEFG}
    EFG9df = pd.DataFrame.from_dict(EFG)
    EFG9df = EFG9df.shift(1)
    EFG9DB = pd.concat([EFG9DB, EFG9df])

TOV9DB = pd.DataFrame()
rollingTOV = []
for team in db['Team'].unique():
    teamRollingTOV = db[db['Team'] == team]['TOV%'].rolling(9).mean()
    rollingTOV.append(teamRollingTOV)
    TOV = {"Rolling9TOV":teamRollingTOV}
    TOV9df = pd.DataFrame.from_dict(TOV)
    TOV9df = TOV9df.shift(1)
    TOV9DB = pd.concat([TOV9DB, TOV9df])
    
ORB9DB = pd.DataFrame()
rollingORB = []
for team in db['Team'].unique():
    teamRollingORB = db[db['Team'] == team]['ORB%'].rolling(9).mean()
    rollingORB.append(teamRollingORB)
    ORB = {"Rolling9ORB":teamRollingORB}
    ORB9df = pd.DataFrame.from_dict(ORB)
    ORB9df = ORB9df.shift(1)
    ORB9DB = pd.concat([ORB9DB, ORB9df])

FTR9DB = pd.DataFrame()
rollingFTR = []
for team in db['Team'].unique():
    teamRollingFTR = db[db['Team'] == team]['FTR'].rolling(9).mean()
    rollingFTR.append(teamRollingFTR)
    FTR = {"Rolling9FTR":teamRollingFTR}
    FTR9df = pd.DataFrame.from_dict(FTR)
    FTR9df = FTR9df.shift(1)
    FTR9DB = pd.concat([FTR9DB, FTR9df])

Pace9DB = pd.DataFrame()
rollingPace = []
for team in db['Team'].unique():
    teamRollingPace = db[db['Team'] == team]['Pace'].rolling(9).mean()
    rollingPace.append(teamRollingPace)
    PACE = {"Rolling9Pace":teamRollingPace}
    PACE9df = pd.DataFrame.from_dict(PACE)
    PACE9df = PACE9df.shift(1)
    Pace9DB = pd.concat([Pace9DB, PACE9df])
    
db = pd.concat([db, EFG9DB, TOV9DB, ORB9DB, FTR9DB, Pace9DB], axis =1)

In [6]:
### 3 Game rolling averages

EFG3DB = pd.DataFrame()
rollingEFG = []
for team in db['Team'].unique():
    teamRollingEFG = db[db['Team'] == team]['EFG%'].rolling(3).mean()
    rollingEFG.append(teamRollingEFG)
    EFG = {"Rolling3EFG":teamRollingEFG}
    EFG3df = pd.DataFrame.from_dict(EFG)
    EFG3df = EFG3df.shift(1)
    EFG3DB = pd.concat([EFG3DB, EFG3df])

TOV3DB = pd.DataFrame()
rollingTOV = []
for team in db['Team'].unique():
    teamRollingTOV = db[db['Team'] == team]['TOV%'].rolling(3).mean()
    rollingTOV.append(teamRollingTOV)
    TOV = {"Rolling3TOV":teamRollingTOV}
    TOV3df = pd.DataFrame.from_dict(TOV)
    TOV3df = TOV3df.shift(1)
    TOV3DB = pd.concat([TOV3DB, TOV3df])
    
ORB3DB = pd.DataFrame()
rollingORB = []
for team in db['Team'].unique():
    teamRollingORB = db[db['Team'] == team]['ORB%'].rolling(3).mean()
    rollingORB.append(teamRollingORB)
    ORB = {"Rolling3ORB":teamRollingORB}
    ORB3df = pd.DataFrame.from_dict(ORB)
    ORB3df = ORB3df.shift(1)
    ORB3DB = pd.concat([ORB3DB, ORB3df])

FTR3DB = pd.DataFrame()
rollingFTR = []
for team in db['Team'].unique():
    teamRollingFTR = db[db['Team'] == team]['FTR'].rolling(3).mean()
    rollingFTR.append(teamRollingFTR)
    FTR = {"Rolling3FTR":teamRollingFTR}
    FTR3df = pd.DataFrame.from_dict(FTR)
    FTR3df = FTR3df.shift(1)
    FTR3DB = pd.concat([FTR3DB, FTR3df])

Pace3DB = pd.DataFrame()
rollingPace = []
for team in db['Team'].unique():
    teamRollingPace = db[db['Team'] == team]['Pace'].rolling(3).mean()
    rollingPace.append(teamRollingPace)
    PACE = {"Rolling3Pace":teamRollingPace}
    PACE3df = pd.DataFrame.from_dict(PACE)
    PACE3df = PACE3df.shift(1)
    Pace3DB = pd.concat([Pace3DB, PACE3df])
    
db = pd.concat([db, EFG3DB, TOV3DB, ORB3DB, FTR3DB, Pace3DB], axis = 1)

#### EDIT IN EXCEL TO FIX DUPLICATE COLUMNS

In [65]:
### Opp R9s

for i, opp in enumerate(db['Opponent']):
    print('Teams Left:', str(len(db['Opponent'])-i))
    db['Opp R9EFG'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9EFG']
    db['Opp R9TOV'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9TOV']
    db['Opp R9ORB'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9ORB']
    db['Opp R9FTR'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9FTR']
    db['Opp R9Pace'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9Pace']

Teams Left: 2460


C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\helgeson\AppData\Local\Continuum\anaconda3

Teams Left: 2459
Teams Left: 2458
Teams Left: 2457
Teams Left: 2456
Teams Left: 2455
Teams Left: 2454
Teams Left: 2453
Teams Left: 2452
Teams Left: 2451
Teams Left: 2450
Teams Left: 2449
Teams Left: 2448
Teams Left: 2447
Teams Left: 2446
Teams Left: 2445
Teams Left: 2444
Teams Left: 2443
Teams Left: 2442
Teams Left: 2441
Teams Left: 2440
Teams Left: 2439
Teams Left: 2438
Teams Left: 2437
Teams Left: 2436
Teams Left: 2435
Teams Left: 2434
Teams Left: 2433
Teams Left: 2432
Teams Left: 2431
Teams Left: 2430
Teams Left: 2429
Teams Left: 2428
Teams Left: 2427
Teams Left: 2426
Teams Left: 2425
Teams Left: 2424
Teams Left: 2423
Teams Left: 2422
Teams Left: 2421
Teams Left: 2420
Teams Left: 2419
Teams Left: 2418
Teams Left: 2417
Teams Left: 2416
Teams Left: 2415
Teams Left: 2414
Teams Left: 2413
Teams Left: 2412
Teams Left: 2411
Teams Left: 2410
Teams Left: 2409
Teams Left: 2408
Teams Left: 2407
Teams Left: 2406
Teams Left: 2405
Teams Left: 2404
Teams Left: 2403
Teams Left: 2402
Teams Left: 24

In [8]:
### Opp R3s

for i, opp in enumerate(db['Opponent']):
    db['Opp R3EFG'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling3EFG']
    db['Opp R3TOV'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling3TOV']
    db['Opp R3ORB'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling3ORB']
    db['Opp R3FTR'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling3FTR']
    db['Opp R3Pace'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling3Pace']

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\helgeson\AppDat

In [ ]:
db.to_csv('2017stats.csv', index = False)

In [3]:
db['Odds'] = np.zeros(len(db['Date']))
db['Line'] = np.zeros(len(db['Date']))

moneydb = pd.read_csv('MLOdds.csv')
spreaddb = pd.read_csv('Spreads.csv')

for i, date in enumerate(db['Date']):
    
    db['Odds'][i] = moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds']
    print("Team:", db['Team'][i], "Odds:", moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds'])
    db['Line'][i] = spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][i])]['Spread']
    print("Team:", db['Team'][i], "Spread:", spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][i])]['Spread'])

/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Team: CLE Odds: 4923   -470
Name: ML Odds, dtype: int64
Team: CLE Spread: 4923   -9.0
Name: Spread, dtype: float64
Team: CLE Odds: 4928   -140
Name: ML Odds, dtype: int64
Team: CLE Spread: 4928   -2.5
Name: Spread, dtype: float64
Team: CLE Odds: 4949   -1100
Name: ML Odds, dtype: int64
Team: CLE Spread: 4949   -12.0
Name: Spread, dtype: float64
Team: CLE Odds: 4963   -475
Name: ML Odds, dtype: int64
Team: CLE Spread: 4963   -9.0
Name: Spread, dtype: float64
Team: CLE Odds: 4983   -550
Name: ML Odds, dtype: int64
Team: CLE Spread: 4983   -9.5
Name: Spread, dtype: float64
Team: CLE Odds: 4994   -2000
Name: ML Odds, dtype: int64
Team: CLE Spread: 4994   -14.0
Name: Spread, dtype: float64
Team: CLE Odds: 5007   -420
Name: ML Odds, dtype: int64
Team: CLE Spread: 5007   -8.5
Name: Spread, dtype: float64
Team: CLE Odds: 5022   -300
Name: ML Odds, dtype: int64
Team: CLE Spread: 5022   -6.5
Name: Spread, dtype: float64
Team: CLE Odds: 5035   -360
Name: ML Odds, dtype: int64
Team: CLE Spread: 50

In [4]:
db.to_csv('2017stats.csv', index = False)

### Getting scores, efficiencies:

In [23]:
def get_scores():
    team_abbrevs = {
    'Cleveland Cavaliers': 'CLE',
    'Boston Celtics':'BOS',
    'Washington Wizards':'WAS',
    'Charlotte Hornets':'CHO',
    'Minnesota Timberwolves':'MIN',
    'Dallas Mavericks':'DAL',
    'Milwaukee Bucks':'MIL',
    'Philadelphia 76ers':'PHI',
    'Phoenix Suns':'PHO',
    'Los Angeles Lakers':'LAL',
    'Utah Jazz':'UTA',
    'Sacramento Kings':'SAC',
    'New York Knicks':'NYK',
    'New Orleans Pelicans':'NOP',
    'Detroit Pistons':'DET',
    'Atlanta Hawks':'ATL',
    'Chicago Bulls':'CHI',
    'Miami Heat':'MIA',
    'Memphis Grizzlies':'MEM',
    'Golden State Warriors':'GSW',
    'Denver Nuggets':'DEN',
    'Brooklyn Nets':'BRK',
    'Los Angeles Clippers':'LAC',
    'Portland Trail Blazers':'POR',
    'Indiana Pacers':'IND',
    'San Antonio Spurs':'SAS',
    'Houston Rockets':'HOU',
    'Oklahoma City Thunder':'OKC',
    'Toronto Raptors':'TOR',
    'Orlando Magic':'ORL'
    }
    
    all_points = []
    all_opp_points = []
    
    for team in list(team_abbrevs.values()):
        print("Current team: " + team)
        url = f'https://www.basketball-reference.com/teams/{team}/2017_games.html'
        res = requests.get(url, 'html_parser')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'html.parser')
        
        # Get just regular season 
        reg_season = soup.find('table',{'id':'games'})
        
        # Find regular season game scors
        souppoints = reg_season.find_all(attrs = {'data-stat':'pts'})
        soupopp = reg_season.find_all(attrs = {'data-stat':'opp_pts'})
        teampoints = [entry.get_text() for entry in souppoints if not entry.get_text().startswith('T')]
        opppoints = [entry.get_text() for entry in soupopp if not entry.get_text().startswith('O')]
        teampoints = list(map(lambda x: int(x), teampoints))
        opppoints = list(map(lambda x: int(x), opppoints))
        all_points.append(teampoints)
        all_opp_points.append(opppoints)
        
    return all_points, all_opp_points

gamepoints = get_scores()
team_points = [item for sublist in gamepoints[0] for item in sublist]
opp_points = [item for sublist in gamepoints[1] for item in sublist]
points = {"Team Points":team_points, 'Opp Points': opp_points}

scores = pd.DataFrame.from_dict(points)
updatedb = pd.concat([db, scores], axis = 1)
updatedb.to_csv('2017stats.csv', index = False)

Current team: CLE
Current team: BOS
Current team: WAS
Current team: CHO
Current team: MIN
Current team: DAL
Current team: MIL
Current team: PHI
Current team: PHO
Current team: LAL
Current team: UTA
Current team: SAC
Current team: NYK
Current team: NOP
Current team: DET
Current team: ATL
Current team: CHI
Current team: MIA
Current team: MEM
Current team: GSW
Current team: DEN
Current team: BRK
Current team: LAC
Current team: POR
Current team: IND
Current team: SAS
Current team: HOU
Current team: OKC
Current team: TOR
Current team: ORL


In [25]:
db = updatedb

In [26]:
db.head()

,Team,Game Number,Location,Result,Date,Season,Opponent,EFG%,TOV%,ORB%,...,Rolling3EFG,Rolling3TOV,Rolling3ORB,Rolling3FTR,Rolling3Pace,Odds,Line,Implied Prob,Team Points,Opp Points
0,CLE,1,1,1,20161025,2016-2017,NYK,0.548,0.120,0.275,...,NaN,NaN,NaN,NaN,NaN,-470,-9.0,82.456140,117,88
1,CLE,2,0,1,20161028,2016-2017,TOR,0.494,0.161,0.214,...,NaN,NaN,NaN,NaN,NaN,-140,-2.5,58.333333,94,91
2,CLE,3,1,1,20161029,2016-2017,ORL,0.472,0.082,0.260,...,NaN,NaN,NaN,NaN,NaN,-1100,-12.0,91.666667,105,99
3,CLE,4,1,1,20161101,2016-2017,HOU,0.614,0.102,0.364,...,0.504667,0.121,0.249667,0.200667,96.766667,-475,-9.0,82.608696,128,120
4,CLE,5,1,1,20161103,2016-2017,BOS,0.571,0.074,0.194,...,0.526667,0.115,0.279333,0.255333,94.400000,-550,-9.5,84.615385,128,122


In [27]:
db.to_csv('2017stats.csv', index = False)

In [28]:
db['Cover'] = np.zeros(len(db['Date']))
for i, n in enumerate(db['Team Points']):
    if n + db['Line'][i] > db['Opp Points'][i]:
        db['Cover'][i] = 1
    elif n + db['Line'][i] < db['Opp Points'][i]:
        db['Cover'][i] = -1
    else:
        db['Cover'][i] = 0

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
db.to_csv('2017stats.csv', index = False)

Now I have odds, lines, covers, S/U results, and cover results. I need at least defensive officiency, I will pull offensive efficiency as well but that might override the four factor stats too strongly if put into the model 

In [30]:
def get_effs(team, opp, loc, date):

    if loc == 1:
        url = f'https://www.basketball-reference.com/boxscores/{date}0{team}.html'
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
        teamstats = soup.find('table',{'id':f'box_{team.lower()}_advanced'})
        oppstats = soup.find('table', {'id':f'box_{opp.lower()}_advanced'})
        Deff = teamstats.find_all('td', {'data-stat':'def_rtg'})[-1].get_text()
        Oeff = teamstats.find_all('td', {'data-stat':'off_rtg'})[-1].get_text()
        Opp_Deff = oppstats.find_all('td', {'data-stat':'def_rtg'})[-1].get_text()
        Opp_Oeff = oppstats.find_all('td', {'data-stat':'off_rtg'})[-1].get_text()
    else:
        url = f'https://www.basketball-reference.com/boxscores/{date}0{opp}.html'
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
        teamstats = soup.find('table',{'id':f'box_{team.lower()}_advanced'})
        oppstats = soup.find('table', {'id':f'box_{opp.lower()}_advanced'})
        Deff = teamstats.find_all('td', {'data-stat':'def_rtg'})[-1].get_text()
        Oeff = teamstats.find_all('td', {'data-stat':'off_rtg'})[-1].get_text()
        Opp_Deff = oppstats.find_all('td', {'data-stat':'def_rtg'})[-1].get_text()
        Opp_Oeff = oppstats.find_all('td', {'data-stat':'off_rtg'})[-1].get_text()
        
    return float(Deff), float(Oeff), float(Opp_Deff), float(Opp_Oeff)

In [31]:
get_effs('CLE', 'TOR',0, '20161028')

(96.9, 100.1, 100.1, 96.9)

In [32]:
db = pd.read_csv('2017stats.csv')

In [33]:
db['Oeff'] = np.zeros(len(db['Date']))
db['Deff'] = np.zeros(len(db['Date']))
db['Opp Oeff'] = np.zeros(len(db['Date']))
db['Opp Deff'] = np.zeros(len(db['Date']))

In [34]:
for i in range(len(db['Date'])):
    effs = get_effs(db['Team'][i], db['Opponent'][i], db['Location'][i], db['Date'][i])
    db['Deff'][i] = effs[0]
    db['Oeff'][i] = effs[1]
    db['Opp Deff'][i] = effs[2]
    db['Opp Oeff'][i] = effs[3]

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [35]:
db.to_csv('2017stats.csv', index = False)

In [36]:
### 9 game rolling efficiencies

Oeff9DB = pd.DataFrame()
rollingOeff= []
for team in db['Team'].unique():
    teamRollingOeff = db[db['Team'] == team]['Oeff'].rolling(9).mean()
    rollingOeff.append(teamRollingOeff)
    OEFF = {"Rolling9OEff":teamRollingOeff}
    OEFF9df = pd.DataFrame.from_dict(OEFF)
    OEFF9df = OEFF9df.shift(1)
    Oeff9DB = pd.concat([Oeff9DB, OEFF9df])

Deff9DB = pd.DataFrame()
rollingDeff = []
for team in db['Team'].unique():
    teamRollingDeff = db[db['Team'] == team]['Deff'].rolling(9).mean()
    rollingDeff.append(teamRollingDeff)
    DEFF = {"Rolling9DEff":teamRollingDeff}
    DEFF9df = pd.DataFrame.from_dict(DEFF)
    DEFF9df = DEFF9df.shift(1)
    Deff9DB = pd.concat([Deff9DB, DEFF9df])
    

db = pd.concat([db, Oeff9DB, Deff9DB], axis =1)

In [64]:
db.head(20)

,Team,Game Number,Location,Result,Date,Season,Opponent,EFG%,TOV%,ORB%,...,Implied Proba,Team Points,Opp Points,Cover,Oeff,Deff,Opp Oeff,Opp Deff,Rolling9OEff,Rolling9DEff
0,CLE,1,1,1,20161025,2016-2017,NYK,0.548,0.120,0.275,...,82.456140,117,88,1.0,117.1,88.1,88.1,117.1,NaN,NaN
1,CLE,2,0,1,20161028,2016-2017,TOR,0.494,0.161,0.214,...,58.333333,94,91,1.0,100.1,96.9,96.9,100.1,NaN,NaN
2,CLE,3,1,1,20161029,2016-2017,ORL,0.472,0.082,0.260,...,91.666667,105,99,-1.0,108.8,102.6,102.6,108.8,NaN,NaN
3,CLE,4,1,1,20161101,2016-2017,HOU,0.614,0.102,0.364,...,82.608696,128,120,-1.0,138.0,129.3,129.3,138.0,NaN,NaN
4,CLE,5,1,1,20161103,2016-2017,BOS,0.571,0.074,0.194,...,84.615385,128,122,-1.0,128.8,122.8,122.8,128.8,NaN,NaN
5,CLE,6,0,1,20161105,2016-2017,PHI,0.484,0.139,0.216,...,95.238095,102,101,-1.0,99.7,98.7,98.7,99.7,NaN,NaN
6,CLE,7,1,0,20161108,2016-2017,ATL,0.429,0.052,0.250,...,80.769231,106,110,-1.0,108.6,112.7,112.7,108.6,NaN,NaN
7,CLE,8,0,1,20161111,2016-2017,WAS,0.500,0.133,0.265,...,75.000000,105,94,1.0,107.8,96.5,96.5,107.8,NaN,NaN
8,CLE,9,1,1,20161113,2016-2017,CHO,0.500,0.084,0.255,...,78.260870,100,93,-1.0,108.1,100.6,100.6,108.1,NaN,NaN
9,CLE,10,1,1,20161115,2016-2017,TOR,0.608,0.151,0.263,...,70.149254,121,117,-1.0,122.2,118.2,118.2,122.2,113.000000,105.355556


In [37]:
### 3 game rolling efficiencies

OeffDB = pd.DataFrame()
rollingOeff= []
for team in db['Team'].unique():
    teamRollingOeff = db[db['Team'] == team]['Oeff'].rolling(3).mean()
    rollingOeff.append(teamRollingOeff)
    OEFF = {"Rolling3OEff":teamRollingOeff}
    OEFFdf = pd.DataFrame.from_dict(OEFF)
    OEFFdf = OEFFdf.shift(1)
    OeffDB = pd.concat([OeffDB, OEFFdf])

DeffDB = pd.DataFrame()
rollingDeff = []
for team in db['Team'].unique():
    teamRollingDeff = db[db['Team'] == team]['Deff'].rolling(3).mean()
    rollingDeff.append(teamRollingDeff)
    DEFF = {"Rolling3DEff":teamRollingDeff}
    DEFFdf = pd.DataFrame.from_dict(DEFF)
    DEFFdf = DEFFdf.shift(1)
    DeffDB = pd.concat([DeffDB, DEFFdf])
    

db = pd.concat([db, OeffDB, DeffDB], axis =1)

In [69]:
db.to_csv('2017stats.csv', index = False)

In [11]:
db = pd.read_csv('2017stats.csv')

In [38]:
db['Opp R9 Oeff'] = np.zeros(len(db['Date']))
db['Opp R9 Deff'] = np.zeros(len(db['Date']))
db['Opp R3 Oeff'] = np.zeros(len(db['Date']))
db['Opp R3 Deff'] = np.zeros(len(db['Date']))

for i, opp in enumerate(db['Opponent']):
    print('Teams Left:', str(len(db['Opponent'])-i))
    db['Opp R9 Oeff'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9OEff']
    db['Opp R9 Deff'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling9DEff']
    db['Opp R3 Oeff'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling3OEff']
    db['Opp R3 Deff'][i] = db[(db['Team'] == opp) & (db['Game Number'] == db['Game Number'][i])]['Rolling3DEff']

Teams Left: 2460
Teams Left: 2459


C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while w

Teams Left: 2458
Teams Left: 2457
Teams Left: 2456
Teams Left: 2455
Teams Left: 2454
Teams Left: 2453
Teams Left: 2452
Teams Left: 2451
Teams Left: 2450
Teams Left: 2449
Teams Left: 2448
Teams Left: 2447
Teams Left: 2446
Teams Left: 2445
Teams Left: 2444
Teams Left: 2443
Teams Left: 2442
Teams Left: 2441
Teams Left: 2440
Teams Left: 2439
Teams Left: 2438
Teams Left: 2437
Teams Left: 2436
Teams Left: 2435
Teams Left: 2434
Teams Left: 2433
Teams Left: 2432
Teams Left: 2431
Teams Left: 2430
Teams Left: 2429
Teams Left: 2428
Teams Left: 2427
Teams Left: 2426
Teams Left: 2425
Teams Left: 2424
Teams Left: 2423
Teams Left: 2422
Teams Left: 2421
Teams Left: 2420
Teams Left: 2419
Teams Left: 2418
Teams Left: 2417
Teams Left: 2416
Teams Left: 2415
Teams Left: 2414
Teams Left: 2413
Teams Left: 2412
Teams Left: 2411
Teams Left: 2410
Teams Left: 2409
Teams Left: 2408
Teams Left: 2407
Teams Left: 2406
Teams Left: 2405
Teams Left: 2404
Teams Left: 2403
Teams Left: 2402
Teams Left: 2401
Teams Left: 24

In [39]:
db.to_csv('2017stats.csv', index = False)

In [54]:
db17 = pd.read_csv('2017stats.csv')

In [55]:
db18 = pd.read_csv('2018stats.csv')

In [56]:
db1718 = pd.concat([db18, db17])

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [57]:
db1718 = db1718.drop(['Unnamed: 0'], axis = 1).reset_index(drop = True)

In [59]:
db1718.to_csv('2017_2018stats.csv')